<a href="https://colab.research.google.com/github/kulkarnivinayv/EEG-and-ChronoNet/blob/main/3_8_Chrononet_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install mne
!wget https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/fshy54ypyh-1.zip -O data.zip
!unzip data.zip

In [2]:
from glob import glob
import scipy.io
import numpy as np
import mne

In [3]:
IDD_data_path='/content/Data/CleanData/CleanData_TDC/Rest'
TDC_data_path='/content/Data/Data/CleanData/Data/Data/CleanData/CleanData_IDD/Rest'
!rm '/content/Data/Data/CleanData/Data/Data/CleanData/CleanData_IDD/Rest/NDS001_Rest_CD(1).mat'

In [17]:
from glob import glob
import scipy.io
for idd in glob(IDD_data_path+'/*.mat'):
  #print(idd)
  data = scipy.io.loadmat(idd)
  break

## **Check whats inside the data.**

In [18]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'clean_data'])

In [19]:
data = data['clean_data']
data.shape

(14, 15360)

***There are 14 channels and length of data is 15360 (fs = 128Hz)***

***Now need to convert this .mat file into MNE format.***

In [4]:
def convertmat2mne(data):
  ch_names = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
  ch_types = ['eeg'] * 14
  sampling_freq=128
  info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
  info.set_montage('standard_1020')
  data=mne.io.RawArray(data, info)
  data.set_eeg_reference()
  data.filter(l_freq=1,h_freq=30)
  epochs=mne.make_fixed_length_epochs(data,duration=4,overlap=0)
  return epochs.get_data()

In [20]:
%%capture
idd_subject=[]
for idd in glob(IDD_data_path+'/*.mat'):
  data=scipy.io.loadmat(idd)['clean_data']
  data=convertmat2mne(data)
  idd_subject.append(data)

In [21]:
%%capture
tdc_subject=[]
for tdc in glob(TDC_data_path+'/*.mat'):
  data=scipy.io.loadmat(tdc)['clean_data']
  data=convertmat2mne(data)
  tdc_subject.append(data)

***Now the dataset is created 7 subjects of IDD and 7 subjects of TDC.***


***Lets now create labels for this data.***

In [26]:
control_epochs_labels=[len(i)*[0] for i in tdc_subject]
patients_epochs_labels=[len(i)*[1] for i in idd_subject]
print(len(control_epochs_labels),len(patients_epochs_labels))

7 7


In [30]:
data_list=tdc_subject+idd_subject
label_list=control_epochs_labels+patients_epochs_labels
groups_list=[[i]*len(j) for i, j in enumerate(data_list)]
print(len(data_list),len(label_list),len(groups_list))

14 14 14


In [36]:
from sklearn.model_selection import GroupKFold,LeaveOneGroupOut
from sklearn.preprocessing import StandardScaler
gkf=GroupKFold()
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import StandardScaler
#https://stackoverflow.com/questions/50125844/how-to-standard-scale-a-3d-matrix
class StandardScaler3D(BaseEstimator,TransformerMixin):
    #batch, sequence, channels
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape( -1,X.shape[2])).reshape(X.shape)

In [31]:
import numpy as np
data_array=np.concatenate(data_list)
label_array=np.concatenate(label_list)
group_array=np.concatenate(groups_list)
data_array=np.moveaxis(data_array,1,2)

print(data_array.shape,label_array.shape,group_array.shape)

(420, 512, 14) (420,) (420,)




1.   420 are number of epochs/trials, 
2.   512 are samples or it is sequence length
3.   14 are electrodes



In [38]:
accuracy=[]
for train_index, val_index in gkf.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler3D()
    train_features=scaler.fit_transform(train_features)
    val_features=scaler.transform(val_features)
   

    break

In [39]:
train_features.shape

(330, 512, 14)

In [37]:
from tensorflow.keras.layers import Input,Dense,concatenate,Flatten,GRU,Conv1D
from tensorflow.keras.models import Model
#resource:https://github.com/dll-ncai/eeg_pre-diagnostic_screening/blob/master/code/chrononet/chrono.py

In [33]:
def block(input):
  conv1 = Conv1D(32, 2, strides=2,activation='relu',padding="same")(input)
  conv2 = Conv1D(32, 4, strides=2,activation='relu',padding="causal")(input)
  conv3 = Conv1D(32, 8, strides=2,activation='relu',padding="causal")(input)
  x = concatenate([conv1,conv2,conv3],axis=2)
  return x

In [50]:
input= Input(shape=(512,14))
block1=block(input)
block2=block(block1)
block3=block(block2)
block3.shape

TensorShape([None, 64, 96])

In [51]:
gru_out1 = GRU(32,activation='tanh',return_sequences=True)(block3)
gru_out2 = GRU(32,activation='tanh',return_sequences=True)(gru_out1)
gru_out = concatenate([gru_out1,gru_out2],axis=2)
gru_out3 = GRU(32,activation='tanh',return_sequences=True)(gru_out)
gru_out = concatenate([gru_out1,gru_out2,gru_out3])
gru_out4 = GRU(32,activation='tanh')(gru_out)

In [52]:
predictions = Dense(1,activation='sigmoid')(gru_out4)
model = Model(inputs=input, outputs=predictions)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [53]:
model.fit(train_features, train_labels, epochs= 10, batch_size = 128, validation_data = (val_features,val_labels))

Epoch 1/10
3/3 [==============================] - 15s 1s/step - loss: 0.6774 - accuracy: 0.5424 - val_loss: 0.7248 - val_accuracy: 0.3333
Epoch 2/10
3/3 [==============================] - 1s 486ms/step - loss: 0.5997 - accuracy: 0.7515 - val_loss: 0.5395 - val_accuracy: 0.9333
Epoch 3/10
3/3 [==============================] - 2s 498ms/step - loss: 0.4456 - accuracy: 0.8909 - val_loss: 0.2903 - val_accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 2s 507ms/step - loss: 0.2347 - accuracy: 0.9848 - val_loss: 0.0964 - val_accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 2s 494ms/step - loss: 0.0819 - accuracy: 1.0000 - val_loss: 0.0302 - val_accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 2s 499ms/step - loss: 0.0248 - accuracy: 1.0000 - val_loss: 0.0186 - val_accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 2s 498ms/step - loss: 0.0110 - accuracy: 1.0000 - val_loss: 0.0066 - val_accuracy: 1.0000
Epoch 8/10
3/3 [======

In [54]:
model.evaluate(val_features,val_labels)

3/3 [==============================] - 0s 54ms/step - loss: 0.0028 - accuracy: 1.0000


[0.002780498005449772, 1.0]